In [1]:
!pip install -q  pydantic sortedcontainers
!pip install -q  ducklake_delta_exporter
!pip install -q  duckdb --upgrade
!pip install -q -i  https://test.pypi.org/simple/ --pre pyiceberg
import sys
sys.exit(0)

sys.exit called with value 0. The interpreter will be restarted.


<mark>**Generate Data**</mark>

In [2]:
import os
import urllib.request
import pyarrow.parquet as pq
import pyarrow as pa

local_path = "/lakehouse/default/Files/data"
os.makedirs(local_path, exist_ok=True)

urls = [
    f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{y}-{m:02d}.parquet"
    for y in range(2020, 2022)
    for m in range(1, 5)
]

local_files = []
for url in urls:
    filename = os.path.join(local_path, os.path.basename(url))
    if not os.path.exists(filename):
        print(f"Downloading {url} ...")
        urllib.request.urlretrieve(url, filename)
    local_files.append(filename)

tables = []
for file in local_files:
    table = pq.read_table(file)
    tables.append(table)
data = pa.concat_tables(tables, promote_options="default")

**<mark>Write Delta</mark>**

In [3]:
%%time
from deltalake import write_deltalake
write_deltalake('/lakehouse/default/Tables/simple/delta',data,mode='overwrite' , engine ='rust', storage_options={"allow_unsafe_rename":"true"})

rr /synfs/lakehouse/default/Tables/simple/delta/_delta_log/_commit_5e774aa1-edd5-4a52-8f42-9f1a6067a8b4.json.tmp -> /synfs/lakehouse/default/Tables/simple/delta/_delta_log/00000000000000000000.json
CPU times: user 10.9 s, sys: 448 ms, total: 11.3 s
Wall time: 20.7 s


**<mark>write Iceberg</mark>**

In [4]:
%%time
from   pyiceberg.catalog.sql import SqlCatalog
catalog = SqlCatalog("default", **{"uri" : "sqlite://///lakehouse/default/Files/pyiceberg.db","warehouse" : "/lakehouse/default/Tables"},)
catalog.create_namespace_if_not_exists("simple")
table = catalog.create_table_if_not_exists("simple.iceberg",schema=data.schema )
table.overwrite(data)
import sqlite3
sqlite3.connect("/lakehouse/default/Files/pyiceberg.db").close()

/home/trusted-service-user/jupyter-env/python3.11/lib/python3.11/site-packages/pyiceberg/table/__init__.py:695: UserWarning: Delete operation did not match any records
  warnings.warn("Delete operation did not match any records")


CPU times: user 14.7 s, sys: 662 ms, total: 15.4 s
Wall time: 16.4 s


**<mark>write ducklake</mark>**

In [5]:
%%time
import duckdb
con = duckdb.connect()
con.sql("force install ducklake from core_nightly")
con.sql(f""" ATTACH or replace 'ducklake:/lakehouse/default/Files/meta.db' AS lake (DATA_PATH '/lakehouse/default/Tables') ; USE lake ; """)
con.sql(f"""  create schema if not exists simple ; use simple ; """)
con.sql(" CALL lake.set_option('parquet_row_group_size', 122880*20); ")
con.sql(" create or replace table ducklake as select * from data ")
con.close()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 13.3 s, sys: 1.29 s, total: 14.6 s
Wall time: 10.1 s


In [6]:
from ducklake_delta_exporter import generate_latest_delta_log
generate_latest_delta_log('/lakehouse/default/Files/meta.db')

✅ simple.ducklake: Delta log written v0 (DuckLake snapshot: 2)
Delta export finished.
